In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
import pickle

from google.cloud import storage
from google.colab import auth
from apiclient.http import MediaIoBaseDownload
from googleapiclient.discovery import build
auth.authenticate_user()
gcs_service = build('storage', 'v1')

In [2]:
#read dataset csv from bucket
with open('/tmp/gsutil_download.csv', 'wb') as f:
  # Download the file from a given Google Cloud Storage bucket.
  request = gcs_service.objects().get_media(bucket='malmem-dataset',
                                            object='malmem_dataset.csv')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    _, done = media.next_chunk()

df = pd.read_csv('/tmp/gsutil_download.csv')



In [3]:
X = df.copy()

X

,Category,pslist.nproc,pslist.nppid,pslist.avg_threads,pslist.nprocs64bit,pslist.avg_handlers,dlllist.ndlls,dlllist.avg_dlls_per_proc,handles.nhandles,handles.avg_handles_per_proc,...,svcscan.kernel_drivers,svcscan.fs_drivers,svcscan.process_services,svcscan.shared_process_services,svcscan.interactive_process_services,svcscan.nactive,callbacks.ncallbacks,callbacks.nanonymous,callbacks.ngeneric,Class
0,Benign,45,17,10.555556,0,202.844444,1694,38.500000,9129,212.302326,...,221,26,24,116,0,121,87,0,8,Benign
1,Benign,47,19,11.531915,0,242.234043,2074,44.127660,11385,242.234043,...,222,26,24,118,0,122,87,0,8,Benign
2,Benign,40,14,14.725000,0,288.225000,1932,48.300000,11529,288.225000,...,222,26,27,118,0,120,88,0,8,Benign
3,Benign,32,13,13.500000,0,264.281250,1445,45.156250,8457,264.281250,...,222,26,27,118,0,120,88,0,8,Benign
4,Benign,42,16,11.452381,0,281.333333,2067,49.214286,11816,281.333333,...,222,26,24,118,0,124,87,0,8,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58591,Ransomware-Shade-fa03be3078d1b9840f06745f160eb...,37,15,10.108108,0,215.486487,1453,39.270270,7973,215.486487,...,221,26,24,116,0,120,86,0,8,Malware
58592,Ransomware-Shade-f56687137caf9a67678cde91e4614...,37,14,9.945946,0,190.216216,1347,36.405405,7038,190.216216,...,221,26,24,116,0,116,88,0,8,Malware
58593,Ransomware-Shade-faddeea111a25da4d0888f3044ae9...,38,15,9.842105,0,210.026316,1448,38.105263,7982,215.729730,...,221,26,24,116,0,120,88,0,8,Malware
58594,Ransomware-Shade-f866c086af2e1d8ebaa6f2c863157...,37,15,10.243243,0,215.513513,1452,39.243243,7974,215.513513,...,221,26,24,116,0,120,87,0,8,Malware


In [4]:
X = X.drop('callbacks.nanonymous', axis=1)
X = X.drop('callbacks.ngeneric', axis=1)
X = X.drop('psxview.not_in_csrss_handles', axis=1)
X = X.drop('psxview.not_in_csrss_handles_false_avg', axis=1)
X = X.drop('psxview.not_in_deskthrd', axis=1)
X = X.drop('psxview.not_in_deskthrd_false_avg', axis=1)
X = X.drop('psxview.not_in_eprocess_pool', axis=1)
X = X.drop('psxview.not_in_eprocess_pool_false_avg', axis=1)
X = X.drop('psxview.not_in_ethread_pool', axis=1)
X = X.drop('psxview.not_in_ethread_pool_false_avg', axis=1)
X = X.drop('psxview.not_in_pslist', axis=1)
X = X.drop('psxview.not_in_pslist_false_avg', axis=1)
X = X.drop('psxview.not_in_pspcid_list', axis=1)
X = X.drop('psxview.not_in_pspcid_list_false_avg', axis=1)
X = X.drop('psxview.not_in_session', axis=1)
X = X.drop('psxview.not_in_session_false_avg', axis=1)
X = X.drop('svcscan.interactive_process_services', axis=1)

X = X.drop('Category', axis=1)
class_map = {'Benign':0, 'Malware':1}
X['Class'] = X['Class'].map(class_map)
Y = X['Class']
X = X.drop('Class', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

In [6]:
clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(X_train, y_train)

In [ ]:
y_pred = clf_svm.predict(X_test)

In [7]:
bucket_name = 'malmem-dataset'
blob_name = 'svm_model.pkl'

storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(blob_name)

blob.upload_from_string(pickle.dumps(clf_svm))

In [8]:
print(accuracy_score(y_test, y_pred))
print(average_precision_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(recall_score(y_test, y_pred))

0.9985778485693156
0.9977846748322369
